In [5]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

# MMM(최고 월평균) 계산 함수 (예시: 2015~2023년 데이터 중 월별 평균의 최대값)
def calculate_mmm(group):
    group['Month'] = group['Date'].dt.month
    mmm = group.groupby('Month')['SST (°C)'].mean().max()
    group['MMM'] = mmm
    return group

# HotSpot 계산 및 DHW 계산 함수
def calculate_dhw(group, window=84):
    sst = group['SST (°C)'].values
    mmm = group['MMM'].values
    # HotSpot: SST - MMM
    hotspot = sst - mmm
    # HotSpot이 1°C 이상인 값만 누적
    hotspot = np.where(hotspot >= 1, hotspot, 0)
    # 12주(84일) 이동합
    dhw = []
    for i in range(len(hotspot)):
        start = max(0, i-window+1)
        # 7로 나눠서 °C-weeks로 변환
        dhw.append(hotspot[start:i+1].sum() / 7)
    group['DHW'] = dhw
    return group

# 파일 불러오기
file_path = '/content/drive/MyDrive/realistic_ocean_climate_dataset.csv'

try:
    df = pd.read_csv(file_path, encoding='cp949', parse_dates=['Date'])
except FileNotFoundError:
    df = None

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
if df is not None:
    df = df.sort_values(['Location', 'Date'])
    # MMM 계산
    df = df.groupby('Location', group_keys=False).apply(calculate_mmm)
    # DHW 계산
    df = df.groupby('Location', group_keys=False).apply(calculate_dhw, window=84)
    print(df[['Date', 'Location', 'SST (°C)', 'MMM', 'DHW']].to_string(index=False))
else:
    print('File not found. Please upload the dataset file.')


      Date           Location  SST (°C)       MMM      DHW
2015-01-14      Caribbean Sea     28.86 29.890000 0.000000
2015-03-01      Caribbean Sea     30.09 29.890000 0.000000
2015-03-14      Caribbean Sea     28.04 29.890000 0.000000
2015-04-16      Caribbean Sea     30.72 29.890000 0.000000
2015-04-22      Caribbean Sea     28.99 29.890000 0.000000
2015-04-29      Caribbean Sea     27.45 29.890000 0.000000
2015-08-26      Caribbean Sea     26.89 29.890000 0.000000
2015-09-01      Caribbean Sea     28.39 29.890000 0.000000
2016-05-22      Caribbean Sea     31.31 29.890000 0.202857
2016-06-04      Caribbean Sea     30.53 29.890000 0.202857
2016-06-17      Caribbean Sea     28.29 29.890000 0.202857
2016-09-04      Caribbean Sea     29.64 29.890000 0.202857
2016-09-11      Caribbean Sea     26.23 29.890000 0.202857
2016-09-24      Caribbean Sea     26.09 29.890000 0.202857
2016-12-18      Caribbean Sea     28.20 29.890000 0.202857
2017-03-20      Caribbean Sea     29.59 29.890000 0.2028

<ipython-input-6-b7b311a89f41>:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Location', group_keys=False).apply(calculate_mmm)
<ipython-input-6-b7b311a89f41>:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Location', group_keys=False).apply(calculate_dhw, window=84)


In [25]:

df = df.sort_values(['Location', 'Date'])

# 2. 누적합
df['DHW_Cumulative'] = df.groupby('Location')['DHW'].cumsum()

# 3. 이동창 누적합 (12주로 누적합 계산 [산호표백현상에 12주 DHW 합계가 더 효과적임])
window_size = 12
df['DHW_Rolling12'] = (
    df.groupby('Location')['DHW']
    .rolling(window=window_size, min_periods=1)
    .sum()
    .reset_index(level=0, drop=True)
)

print(df[['Date', 'Location', 'DHW', 'DHW_Cumulative', 'DHW_Rolling12']])
# DHW_Rolling과 DHW_Rolling12가 모두 있으면, 예를 들어 DHW_Rolling만 삭제
if 'DHW_Rolling' in df.columns and 'DHW_Rolling12' in df.columns:
    df = df.drop(columns=['DHW_Rolling'])


          Date            Location       DHW  DHW_Cumulative  DHW_Rolling12
2   2015-01-14       Caribbean Sea  0.000000        0.000000       0.000000
9   2015-03-01       Caribbean Sea  0.000000        0.000000       0.000000
11  2015-03-14       Caribbean Sea  0.000000        0.000000       0.000000
16  2015-04-16       Caribbean Sea  0.000000        0.000000       0.000000
17  2015-04-22       Caribbean Sea  0.000000        0.000000       0.000000
18  2015-04-29       Caribbean Sea  0.000000        0.000000       0.000000
36  2015-08-26       Caribbean Sea  0.000000        0.000000       0.000000
37  2015-09-01       Caribbean Sea  0.000000        0.000000       0.000000
77  2016-05-22       Caribbean Sea  0.202857        0.202857       0.202857
79  2016-06-04       Caribbean Sea  0.202857        0.405714       0.405714
81  2016-06-17       Caribbean Sea  0.202857        0.608571       0.608571
93  2016-09-04       Caribbean Sea  0.202857        0.811429       0.811429
94  2016-09-

In [26]:
# 엑셀로 저장
df.to_excel('DHW.NOAA_Rolling_values.xlsx', index=False)

In [28]:
from google.colab import files
files.download('DHW.NOAA_Rolling_values.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>